In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt

# Importing sector DFs

In [120]:
# importing actual energy generation
PATH_TO_DATA = '/Users/R/code/HaukeFock/predict_energy_generation/raw_data/'
companies = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/

## Actual Energy Generation

In [85]:
columns = ['Date', 'Time of day', 'Biomass[MWh]', 'Hydropower[MWh]',
       'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]',
       'Other renewable[MWh]', 'Nuclear[MWh]', 'Fossil brown coal[MWh]',
       'Fossil hard coal[MWh]', 'Fossil gas[MWh]', 'Hydro pumped storage[MWh]',
       'Other conventional[MWh]']
dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company} = pd.DataFrame(columns=columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/energy_generation/{company}/{file}', sep=';')
        exec(f"df_{company} = df_{company}.append(data, ignore_index=True)")
    exec(f"dataframes['df_{company}'] = df_{company}")
    print(f'df_{company} Loaded!!!')

/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_50hertz Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,4,6,9,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_amprion Loaded!!!
df_tennet Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_transnet_bw Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [86]:
# dropping unecessary columns 
df_50hertz['Date'] = df_50hertz['Date'][:140448].append(df_50hertz['Datum'][140448:])
df_50hertz['Time of day'] = df_50hertz['Time of day'][:140448].append(df_50hertz['Uhrzeit'][140448:])

df_tennet['Date'] = df_tennet['Datum'][:70272].append(df_tennet['Date'][70272:])
df_tennet['Time of day'] = df_tennet['Uhrzeit'][:70272].append(df_tennet['Time of day'][70272:])

df_50hertz.drop(columns=['Datum', 'Uhrzeit'], inplace=True)
df_tennet.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

# missing values resume for each area
for key, df in dataframes.items():
    if (df.columns != columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

>>>>>>>>>>>>>> df_50hertz <<<<<<<<<<<<<<

Shape: (210720, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]                0
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                 210720
Fossil brown coal[MWh]            0
Fossil hard coal[MWh]             0
Fossil gas[MWh]                   0
Hydro pumped storage[MWh]         0
Other conventional[MWh]           0
dtype: int64

>>>>>>>>>>>>>> df_amprion <<<<<<<<<<<<<<

Shape: (210720, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]           210720
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                      0
Fossil

In [89]:
numeric_values = ['Biomass[MWh]',
                 'Hydropower[MWh]',
                 'Wind offshore[MWh]',
                 'Wind onshore[MWh]',
                 'Photovoltaics[MWh]',
                 'Other renewable[MWh]',
                 'Nuclear[MWh]',
                 'Fossil brown coal[MWh]',
                 'Fossil hard coal[MWh]',
                 'Fossil gas[MWh]',
                 'Hydro pumped storage[MWh]',
                 'Other conventional[MWh]']

def set_to_numeric(dataframes, numeric_values):
    # sets the features to numerical values
    for column in numeric_values:
        #print(column)
        for key, df in dataframes.items():
            #print(f'{key} => {column}')
            df[column] = df[column].apply(lambda x: str(x).replace(',',''))
            df[column] = pd.to_numeric(df[column], errors='coerce')

set_to_numeric(dataframes, numeric_values)

In [104]:
# join 'Date' and 'Time of Day' into one
def set_datetime(dataframes):
    for df in dataframes.keys():
        print(f'Working on {df}')
        exec(f"{df}['Date'] = pd.to_datetime({df}['Date']).apply(lambda x: x.strftime('%Y-%m-%d '))")
        exec(f"{df}['Time of day'] = pd.to_datetime({df}['Time of day']).apply(lambda x: x.strftime('%H:%M'))")
        exec(f"{df}['Date'] = pd.to_datetime({df}['Date'] + {df}['Time of day'])")
        exec(f"{df}.drop(columns=['Time of day'], inplace=True)")
set_datetime(dataframes)

Working on df_50hertz
Working on df_amprion
Working on df_tennet
Working on df_transnet_bw


## Forcasted Energy generation

In [99]:
# importing actual energy generation
forecasted_columns = ['Date', 'Time of day', 'Total[MWh]', 'Wind offshore[MWh]', 
           'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'Other[MWh]']
forecasted_dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company}_forecasted = pd.DataFrame(columns=forecasted_columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/forcasted_generation/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/forcasted_generation/{company}/{file}', sep=';')
        exec(f"df_{company}_forecasted = df_{company}_forecasted.append(data, ignore_index=True)")
    exec(f"forecasted_dataframes['df_{company}_forecasted'] = df_{company}_forecasted")
    print(f'df_{company}_forecasted Loaded!!!')

df_50hertz_forecasted Loaded!!!
df_amprion_forecasted Loaded!!!
df_tennet_forecasted Loaded!!!
df_transnet_BW_forecasted Loaded!!!


In [100]:
# dropping unecessary columns
df_tennet_forecasted['Date'] = df_tennet_forecasted.Date[:140448].append(df_tennet_forecasted.Datum[140448:151964])\
                                                          .append(df_tennet_forecasted.Date[151964:])
df_tennet_forecasted['Time of day'] = df_tennet_forecasted['Time of day'][:140448].append(df_tennet_forecasted.Uhrzeit[140448:151964])\
                                                          .append(df_tennet_forecasted['Time of day'][151964:])
df_tennet_forecasted.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

# missing values resume for each area
for key, df in forecasted_dataframes.items():
    if (df.columns != forecasted_columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

In [101]:
# missing values resume for each area
for key, df in forecasted_dataframes.items():
    if (df.columns != forecasted_columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

>>>>>>>>>>>>>> df_50hertz_forecasted <<<<<<<<<<<<<<

Shape: (153116, 7)

NaN values: 
Date                  0
Time of day           0
Total[MWh]            0
Wind offshore[MWh]    0
Wind onshore[MWh]     0
Photovoltaics[MWh]    0
Other[MWh]            0
dtype: int64

>>>>>>>>>>>>>> df_amprion_forecasted <<<<<<<<<<<<<<

Shape: (153116, 7)

NaN values: 
Date                       0
Time of day                0
Total[MWh]                 0
Wind offshore[MWh]    153116
Wind onshore[MWh]          0
Photovoltaics[MWh]         0
Other[MWh]                 0
dtype: int64

>>>>>>>>>>>>>> df_tennet_forecasted <<<<<<<<<<<<<<

Shape: (153116, 7)

NaN values: 
Date                  0
Time of day           0
Total[MWh]            0
Wind offshore[MWh]    0
Wind onshore[MWh]     0
Photovoltaics[MWh]    0
Other[MWh]            0
dtype: int64

>>>>>>>>>>>>>> df_transnet_BW_forecasted <<<<<<<<<<<<<<

Shape: (153116, 7)

NaN values: 
Date                       0
Time of day                0
Total[MWh]    

In [102]:
# setting columns to numeric
forecasted_numeric_columns = ['Total[MWh]','Wind offshore[MWh]','Wind onshore[MWh]',
                            'Photovoltaics[MWh]','Other[MWh]']

set_to_numeric(forecasted_dataframes, forecasted_numeric_columns)

# join 'Date' and 'Time of Day' into one
set_datetime(forecasted_dataframes)

## Production capacity

In [166]:
capacity_columns = ['Date', 'Time of day', 'Biomass[MW]', 'Hydropower[MW]',
       'Wind offshore[MW]', 'Wind onshore[MW]', 'Photovoltaics[MW]',
       'Other renewable[MW]', 'Nuclear[MW]', 'Fossil brown coal[MW]',
       'Fossil hard coal[MW]', 'Fossil gas[MW]', 'Hydro pumped storage[MW]',
       'Other conventional[MW]']
capacity_dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company}_capacity = pd.DataFrame(columns=capacity_columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_capacity/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/energy_capacity/{company}/{file}', sep=';')
        exec(f"df_{company}_capacity = df_{company}_capacity.append(data, ignore_index=True)")
    exec(f"capacity_dataframes['df_{company}_capacity'] = df_{company}_capacity")
    print(f'df_{company}_capacity Loaded!!!')

df_50hertz_capacity Loaded!!!
df_amprion_capacity Loaded!!!
df_tennet_capacity Loaded!!!
df_transnet_bw_capacity Loaded!!!


In [167]:
# dropping duplicates
for df in capacity_dataframes.keys():
    exec(f"{df} = {df}.drop(index=[2,5,8]).reset_index(drop=True)")

In [168]:
capacity_columns

['Date',
 'Time of day',
 'Biomass[MW]',
 'Hydropower[MW]',
 'Wind offshore[MW]',
 'Wind onshore[MW]',
 'Photovoltaics[MW]',
 'Other renewable[MW]',
 'Nuclear[MW]',
 'Fossil brown coal[MW]',
 'Fossil hard coal[MW]',
 'Fossil gas[MW]',
 'Hydro pumped storage[MW]',
 'Other conventional[MW]']

In [169]:
capacity_numeric_values = ['Biomass[MW]', 'Hydropower[MW]', 'Wind offshore[MW]', 'Wind onshore[MW]',
                           'Photovoltaics[MW]', 'Other renewable[MW]', 'Nuclear[MW]', 'Fossil brown coal[MW]',
                           'Fossil hard coal[MW]', 'Fossil gas[MW]', 'Hydro pumped storage[MW]', 
                           'Other conventional[MW]']

set_to_numeric(capacity_dataframes, capacity_numeric_values)

In [171]:
set_datetime(capacity_dataframes)

Working on df_50hertz_capacity
Working on df_amprion_capacity
Working on df_tennet_capacity
Working on df_transnet_bw_capacity


# Group by daily energy generation

In [46]:
# group energy generation per day 
for df in dataframes.keys():
    exec(f"{df} = {df}.groupby(by={df}['Date'].dt.date).mean().reset_index()")

In [111]:
# group forcasted energy generation per day 
for df in forecasted_dataframes.keys():
    exec(f"{df} = {df}.groupby(by={df}['Date'].dt.date).mean().reset_index()")

# Creating complete DF

## Actual

In [47]:
df_deutschland = pd.DataFrame()

for df, company in zip(dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland = df_deutschland.append({df}, ignore_index=True)")

df_deutschland.shape

(8772, 14)

In [146]:
df_deutschland[['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]']].head(10)

,Date,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh]
0,2015-01-01,11.989583,1278.427083,75.531250
1,2015-01-02,9.947917,2555.510417,17.677083
2,2015-01-03,11.281250,2308.958333,27.354167
3,2015-01-04,11.354167,2079.041667,49.750000
4,2015-01-05,4.104167,1333.343750,20.187500
5,2015-01-06,6.760417,516.687500,59.968750
6,2015-01-07,9.520833,946.666667,28.885417
7,2015-01-08,11.906250,1667.093750,16.187500
8,2015-01-09,11.010417,2659.114583,14.052083
9,2015-01-10,6.802083,2456.656250,27.500000


## Forcasted

In [115]:
df_deutschland_forecasted = pd.DataFrame()

for df, company in zip(forecasted_dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland_forecasted = df_deutschland_forecasted.append({df}, ignore_index=True)")

df_deutschland_forecasted.shape

(6372, 7)

In [143]:
df_deutschland_forecasted.tail(10)

,Date,Total[MWh],Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],Other[MWh],region
6362,2021-05-14,4006.166667,NaN,61.552083,316.510417,2495.000000,transnet_bw
6363,2021-05-15,3893.791667,NaN,107.656250,174.802083,2764.750000,transnet_bw
6364,2021-05-16,3768.666667,NaN,139.802083,247.354167,2221.125000,transnet_bw
6365,2021-05-17,4666.458333,NaN,236.645833,222.375000,2831.291667,transnet_bw
6366,2021-05-18,4366.166667,NaN,159.531250,259.708333,2690.166667,transnet_bw
6367,2021-05-19,4933.583333,NaN,60.531250,259.812500,3653.250000,transnet_bw
6368,2021-05-20,4545.458333,NaN,53.145833,291.010417,3170.166667,transnet_bw
6369,2021-05-21,4017.208333,NaN,205.083333,152.958333,2585.958333,transnet_bw
6370,2021-05-22,4138.833333,NaN,240.041667,394.343750,1602.041667,transnet_bw
6371,2021-05-23,NaN,NaN,NaN,NaN,NaN,transnet_bw


## Capacity

In [173]:
df_deutschland_capacity = pd.DataFrame()

for df, company in zip(capacity_dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland_capacity = df_deutschland_capacity.append({df}, ignore_index=True)")

df_deutschland_capacity.shape

(28, 14)

# Saving DFs

## Actual

In [56]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

dataframes['df_deutschland'] = df_deutschland

for df in dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation_data/{df}.csv', index=False)")

In [55]:
df = pd.read_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation_data/df_50hertz')
df.head()

,Date,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],region
0,2015-01-01,11.989583,1278.427083,75.531250,50hertz
1,2015-01-02,9.947917,2555.510417,17.677083,50hertz
2,2015-01-03,11.281250,2308.958333,27.354167,50hertz
3,2015-01-04,11.354167,2079.041667,49.750000,50hertz
4,2015-01-05,4.104167,1333.343750,20.187500,50hertz


## Forecasted

In [175]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

forecasted_dataframes['df_deutschland_forecasted'] = df_deutschland_forecasted

for df in forecasted_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/forecasted_generation_data/{df}.csv', index=False)")

KeyError: "['Wind offshore[MW]', 'Photovoltaics[MW]', 'Wind onshore[MW]'] not in index"

## Capacity

In [177]:
final_columns = ['Date', 'Wind offshore[MW]', 'Wind onshore[MW]', 'Photovoltaics[MW]', 'region']

capacity_dataframes['df_deutschland_capacity'] = df_deutschland_capacity

for df in capacity_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_capacity_data/{df}.csv', index=False)")
    

# scratch

In [148]:
df_deutschland.groupby(by=df_deutschland['Date'].dt.date).sum()

AttributeError: Can only use .dt accessor with datetimelike values

In [150]:
pd.to_datetime(df_deutschland['Date'])

0      2015-01-01
1      2015-01-02
2      2015-01-03
3      2015-01-04
4      2015-01-05
          ...    
8767   2020-12-28
8768   2020-12-29
8769   2020-12-30
8770   2020-12-31
8771   2021-01-01
Name: Date, Length: 8772, dtype: datetime64[ns]